<a href="https://colab.research.google.com/github/p-owens/MAI/blob/main/tf_lite_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import numpy as np 
import tensorflow as tf
import pandas as pd

In [7]:
model_name = "tfl_ch36_pct_err4.05_comp0.95.tflite"
channel = 36

In [8]:
ch_names = np.arange(44).tolist()
freqs = np.arange(191.6, 195.9, 0.1).tolist()
for x in range(len(ch_names)):
  ch_names[x] = "Ch %.0f (%.1f)" % (ch_names[x], freqs[x])

In [9]:
#Using a dataset with 1 ch constantly on for the test set
#retreiving both datasets from GitHub



git_xval = ('https://raw.githubusercontent.com/p-owens/MAI/main/const_ch/new_model/test_set/ch{0}/x_val.csv'.format(channel))
git_yval = ('https://raw.githubusercontent.com/p-owens/MAI/main/const_ch/new_model/test_set/ch{0}/y_val.csv'.format(channel))
dfx = pd.read_csv(git_xval,
                  index_col=False,
                  header=None,
                  names=ch_names                                   
                 )
dfy = pd.read_csv(git_yval,
                  index_col=False,
                  header=None,
                  names=ch_names
                 )

#dropping the frequences row for each training sample
dfx = dfx.iloc[1::2]

#drop all rows where df_x has a value != 0
dfx = dfx[dfx[ch_names[channel]] !=0]
dfy = dfy[dfy[ch_names[channel]] !=0]
dfy = dfy[ch_names[channel]]

x_test = dfx.to_numpy()
y_test = dfy.to_numpy()

In [11]:
#loading a previously saved tflite model for evaluation
interpreter = tf.lite.Interpreter(model_path=model_name)

interpreter.allocate_tensors()
input_index = interpreter.get_input_details()[0]["index"]
output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on ever y image in the "test" dataset.
y_pred = []
for test in (x_test):
  # Pre-processing: add batch dimension and convert to float32 to match with
  # the model's input data format.
  test = np.expand_dims(test, axis=0).astype(np.float32)
  interpreter.set_tensor(input_index, test)

  # Run inference.
  interpreter.invoke()

  prediction = interpreter.get_tensor(output_index)[0][0]
  y_pred.append(prediction)

#return mean absolute percentage error
y_pred = np.array(y_pred)
mse = tf.keras.losses.MeanAbsolutePercentageError()                                 #tf.keras.losses.MeanSquaredError()
print("Model percent error: {:.2f}%".format(mse(y_test, y_pred).numpy()))


Model percent error: 4.05%
